In [8]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST 
from torchvision import transforms
from tqdm import tqdm_notebook as tqdm
from jax.experimental import stax, optimizers
from jax import random, grad, jit, vmap, value_and_grad
import numpy as np
from jax.experimental.stax import Conv, Dense, Flatten, Relu, LogSoftmax
import warnings
import time
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [11]:
class Args():
    def __init__(self):
        self.bs = 256
        self.no_classes = 10
        self.key = random.PRNGKey(1)
        self.no_epochs = 5
        self.lr = 0.0001
args = Args()

In [26]:
tsfm = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = MNIST('/tmp/', train=True, download=True, transform=tsfm)
test_dataset = MNIST('/tmp/', train=False, download=True, transform=tsfm)

train_loader = DataLoader(train_dataset, batch_size=args.bs, shuffle=True, collate_fn=None)
test_loader = DataLoader(test_dataset, batch_size=args.bs, shuffle=True, collate_fn=None)

init_fun, model = stax.serial(Conv(4, (3, 3), (2, 2), padding="SAME"), Relu,
                                 Conv(8, (3, 3), (2, 2), padding="SAME"), Relu,
                                 Flatten, Dense(args.no_classes),
                                 LogSoftmax)

_, params = init_fun(args.key, (args.bs, 1, 28, 28))

In [27]:
@jit
def update(params, data, target, opt_state):
    value, grads = value_and_grad(loss)(params, data, target)
    opt_state = opt_update(0, grads, opt_state)
    return get_params(opt_state), opt_state, value

def one_hot(x, k, dtype=np.float32):
    return np.array(x[:, None] == np.arange(k), dtype)

def accuracy(params, loader, bar=False):
    correct_count = 0
    for data, target in tqdm(loader, disable=bar, desc='Evaluating'):
        target = one_hot(np.array(target), args.no_classes)
        target_class = np.argmax(target, axis=1)
        pred_class = np.argmax(model(params, np.array(data)), axis=1)
        correct_count += np.sum(pred_class == target_class)
    return correct_count/len(loader.dataset)

def loss(params, images, targets):
    preds = model(params, images)
    return -np.sum(preds * targets)

def train(opt_state, epochs=10):

    metrics = {'train':[], 'test':[], 'loss':[]}
    params = get_params(opt_state)
    # metrics['train'].append(round(accuracy(params, train_loader), 2))
    # metrics['test'].append(round(accuracy(params, test_loader), 2))
    start = time.time()
    for epoch in tqdm(range(epochs), desc='Training'):
        for data, target in train_loader:
            target = one_hot(np.array(target), args.no_classes)
            params, opt_state, loss = update(params, np.array(data), target, opt_state)
            metrics['loss'].append(loss)
        # metrics['train'].append(round(accuracy(params, train_loader, bar=True), 2))
    end = time.time() - start
    metrics['test'].append(round(accuracy(params, test_loader, bar=True), 2))
    return metrics, end
args.lr = 0.005
opt_init, opt_update, get_params = optimizers.adam(args.lr)
opt_state = opt_init(params)

metrics, end = train(opt_state, epochs=5)
# plot_accuracy(metrics)
print("Max test accuracy: {}".format(max(metrics['test'])))
print("Total time elapsed: {}".format(end))


Max test accuracy: 0.94
Total time elapsed: 44.977105379104614


Max test accuracy: 0.94 <br>
Training time : 45 sec



In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(9216, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))



def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [32]:
device = torch.device("cuda")
model = Net().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.005)
start = time.time()
for epoch in range(5):
        train(model, device, train_loader, optimizer, epoch)
end = time.time() - start
test(model, device, test_loader)
print('time taken: {}'.format(end))

Train Epoch: 0 [22464/60000 (100%)]	Loss: 0.021103
Train Epoch: 1 [22464/60000 (100%)]	Loss: 0.085423
Train Epoch: 2 [22464/60000 (100%)]	Loss: 0.014231
Train Epoch: 3 [22464/60000 (100%)]	Loss: 0.006445
Train Epoch: 4 [22464/60000 (100%)]	Loss: 0.073616

Test set: Average loss: 0.0515, Accuracy: 9855/10000 (99%)

time taken: 42.83162713050842


99 acc, 43 sec